# Image Colorization Using CGAN

## Steps followed

#### 1. Importing Necessary Libraries
#### 2. Fetching The Dataset and Setting Up Input Paths
#### 3. Defining Train and Test DataLoaders¶
#### 4. Modeling the Conditional GAN
#### 5. Defining Helper Functions
#### 6. Initializing The Model
#### 7. Training
#### 8. Visualizing Loss Trajectory
#### 9. Visualizing Predictions

## Step 1. Importing necessary libraries and Setting Device

In [ ]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from skimage.color import rgb2lab, lab2rgb

from tqdm import tqdm

from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Step 2. Fetching The Dataset and Setting Up Input Paths

### Installing fastai for Quickly Getting The COCO Dataset

In [ ]:
!pip install -U fastai
import fastai

### Grabbing The Dataset in the Following Directory Structure
<pre>
.
└── .fastai
    └── data
        └── coco_sample
            └── train_sample
                └── *.jpg (10,000 images in total)</pre>

In [ ]:
from fastai.data.external import untar_data, URLs
coco_path = untar_data(URLs.COCO_SAMPLE)
coco_path = str(coco_path) + "/train_sample"
paths = glob.glob(coco_path + "/*.jpg")
# Setting seed for getting the same data across all train sessions 
np.random.seed(123)
paths_subset = np.random.choice(paths, 10_000, replace=False) # choosing 10000 images randomly
rand_idxs = np.random.permutation(10_000)
train_idxs = rand_idxs[:8000] # choosing the first 8000 as training set
val_idxs = rand_idxs[8000:] # choosing last 2000 as validation set
train_paths = paths_subset[train_idxs]
val_paths = paths_subset[val_idxs]
print(train_paths)

### Previewing The Input Images

In [ ]:
imageCount = 0
fig, ax = plt.subplots(4, 4, figsize=(13,13))
for i in range(4):
    for j in range(4):        
        ax[i, j].imshow(Image.open(train_paths[imageCount]))
        imageCount+=1

## Step 3. Defining Train and Test DataLoaders

In [ ]:
ImageSize = 256
class MakeDataset(Dataset):
    def __init__(self, paths):
        self.transforms = transforms.Compose([
                transforms.Resize((ImageSize, ImageSize),  Image.Resampling.BICUBIC),
                transforms.RandomHorizontalFlip(), # Added after 350th Epoch to see if Results improves
            ])
        self.paths=paths

    def __getitem__(self, i):
        img = Image.open(self.paths[i])
        img = img.convert("RGB")
        img = self.transforms(img)
        img = np.array(img)
        imgInLAB = rgb2lab(img).astype("float32")
        imgInLAB = transforms.ToTensor()(imgInLAB)
        L_array = imgInLAB[[0], ...] / 50. - 1.
        ab_array = imgInLAB[[1, 2], ...] / 110.
        return [L_array, ab_array]
        
    def __len__(self):
        return len(self.paths)
    

#### Making dataloaders with input images transformed to L and ab image space, after resizing to 256x256

In [ ]:
BatchSize, Workers = [16, 4]
train_dl = DataLoader(MakeDataset(paths=train_paths), batch_size=BatchSize, num_workers=Workers, pin_memory=True, shuffle = True)
val_dl = DataLoader(MakeDataset(paths=val_paths), batch_size=BatchSize, num_workers=Workers, pin_memory=True, shuffle = True)


#### Looking at the Transformed Data

##### Helper Function for Converting a batch of Lab images into a batch of RGB images

In [ ]:
def lab_to_rgb(L, ab):  
    L = (L + 1.) * 50.
    ab = ab * 110.
    Lab = torch.cat([L, ab], dim=1).permute(0, 2, 3, 1).cpu().numpy()
    rgb_imgs = []
    for img in Lab:
        img_rgb = lab2rgb(img)
        rgb_imgs.append(img_rgb)
    return np.stack(rgb_imgs, axis=0)

In [ ]:
data = next(iter(train_dl))
L_Array, ab_Array = data[0], data[1]
print(f"L Array Shape : {L_Array.shape}", f"*a*b Array Shape : {ab_Array.shape}",sep='\n')

fig, (ax0, ax1, ax2, ax3) = plt.subplots(1, 4, figsize=(15,10))
ax0.imshow(L_Array[0][0], cmap='gray')
ax0.set_title('L')
ax1.imshow(ab_Array[0][0])
ax1.set_title('a')
ax2.imshow(ab_Array[0][1])
ax2.set_title('b')
ax3.imshow(lab_to_rgb(L_Array,ab_Array)[0])
ax3.set_title('RGB')
plt.show()

## Step 4. Modeling the Conditional GAN

### Generator Model

In [ ]:
class GenBlock(nn.Module):
    def __init__(self, inputs, outputs, down=True, batchNorm=True, dropout=False):
        super(GenBlock,self).__init__()

        if down:
            self.block1 = nn.Conv2d(inputs, outputs, kernel_size=4, stride=2, padding=1, bias=False)
            self.block4 = nn.LeakyReLU(0.2, True)
        else:
            self.block1 = nn.ConvTranspose2d(inputs, outputs, kernel_size=4, stride=2, padding=1, bias=False)
            self.block4 = nn.ReLU(True)
        if batchNorm:
            self.block2 = nn.BatchNorm2d(outputs)
        if dropout:
            self.block3 = nn.Dropout(0.5)

        self.batchNorm = batchNorm
        self.dropout = dropout
    
    def forward(self, x):
        out = self.block1(x)
        if self.batchNorm:
            out = self.block2(out)
        if self.dropout:
            out = self.block3(out)
        out = self.block4(out)
        return out

In [ ]:
class Generator(nn.Module):
    def __init__(self, inputs=1):
        super(Generator,self).__init__()
        
        self.d1=  GenBlock(inputs,64,batchNorm=False)
        self.d2=  GenBlock(64,128)
        self.d3=  GenBlock(128,256)
        self.d4=  GenBlock(256,512)
        self.d5=  GenBlock(512,512)
        self.d6=  GenBlock(512,512)
        self.d7=  GenBlock(512,512)
        self.d8=  nn.Sequential(nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1, bias=False), nn.LeakyReLU(0.2))
        
        
        self.u1 = GenBlock(512,512,False,dropout=True)
        self.u2 = GenBlock(1024,512,False,dropout=True)
        self.u3 = GenBlock(1024,512,False,dropout=True)
        self.u4 = GenBlock(1024,512,False)
        self.u5 = GenBlock(1024,256,False)
        self.u6 = GenBlock(512,128,False)
        self.u7 = GenBlock(256,64,False)
        self.u8 = nn.Sequential(nn.ConvTranspose2d(128, 2, kernel_size=4, stride=2, padding=1, bias=False), nn.Tanh())
        
    
    def forward(self, x):
        dd1 = self.d1(x)
        dd2 = self.d2(dd1)
        dd3 = self.d3(dd2)
        dd4 = self.d4(dd3)
        dd5 = self.d5(dd4)
        dd6 = self.d6(dd5)
        dd7 = self.d7(dd6)
        dd8 = self.d8(dd7)
        uu1 = self.u1(dd8)
        uu2 = self.u2(torch.concat([uu1,dd7],1)) #Skip Connection from dd7 to uu1
        uu3 = self.u3(torch.concat([uu2,dd6],1))
        uu4 = self.u4(torch.concat([uu3,dd5],1))
        uu5 = self.u5(torch.concat([uu4,dd4],1))
        uu6 = self.u6(torch.concat([uu5,dd3],1))
        uu7 = self.u7(torch.concat([uu6,dd2],1))
        uu8 = self.u8(torch.concat([uu7,dd1],1))
        return uu8

#### Generator Model Summary

In [ ]:
!pip install -U torchsummary
from torchsummary import summary

In [ ]:
gen=Generator(1)
summary(gen,(1,ImageSize,ImageSize),BatchSize)

### Discriminator Model

In [ ]:
class DiscBlock(nn.Module):
    def __init__(self, inputs, outputs,  kernel=4, stride=2, padding=1, batchNorm=True, activation=True):
        super(DiscBlock,self).__init__()
        
        self.block1 = nn.Conv2d(inputs, outputs, kernel, stride, padding, bias=not batchNorm)
        if batchNorm: self.block2 = nn.BatchNorm2d(outputs)
        if activation: self.block3 = nn.LeakyReLU(0.2, True)

        self.batchNorm = batchNorm
        self.activation = activation


    def forward(self, x):
        out = self.block1(x)
        if self.batchNorm:
            out = self.block2(out)
        if self.activation:
            out = self.block3(out)
        # print(out.shape)
        return out
        

class Discriminator(nn.Module):
    def __init__(self, inputs=3):
        super(Discriminator,self).__init__()

        self.b1 = DiscBlock(inputs,64,batchNorm=False)
        self.b2 = DiscBlock(64,128)
        self.b3 = DiscBlock(128,256)
        self.b4 = DiscBlock(256,512,stride=1)
        self.b5 = DiscBlock(512,1,stride=1,batchNorm=False,activation=False)
                                
    def forward(self, x):
        #print(x.shape())
        y1 = self.b1(x)
        y2 = self.b2(y1)
        y3 = self.b3(y2)
        y4 = self.b4(y3)
        y5 = self.b5(y4)
        return y5

#### Discriminator Model Summary

In [ ]:
testDiscriminator=Discriminator(3)
summary(testDiscriminator,(3,256,256),16)

## Step 5. Defining Helper Functions

#### For Generating Some Predictions

In [ ]:
def ShowSamples(Model, dl, folder, epoch= -1, SAVE = True,suffix=""):
    data = next(iter(dl))
    L, ab = data[0], data[1]
    L=L.to(device)
    ab=ab.to(device)
    # print(ab.shape)
    #Setting Model to Evaluation Mode.
    Model.eval()
    with torch.no_grad():
        ab_gen = Model(L)
    Model.train()
    real_imgs = lab_to_rgb(L, ab)
    fake_imgs = lab_to_rgb(L, ab_gen.detach())

    fig = plt.figure(figsize=(15, 8))
    for i in range(5):
        ax = plt.subplot(3, 5, i + 1)
        ax.imshow(L[i][0].cpu(), cmap='gray')
        if i==0:
            ax.set_ylabel('Grayscale', size='large')
        ax = plt.subplot(3, 5, i + 1 + 5)
        ax.imshow(fake_imgs[i])
        if i==0:
            ax.set_ylabel('Prediction', size='large')
        ax = plt.subplot(3, 5, i + 1 + 10)
        ax.imshow(real_imgs[i])
        if i==0:
            ax.set_ylabel('Ground Truth', size='large')
    plt.show()
    if SAVE:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        fig.savefig(folder + f"/Results_After_Epoch_{epoch}{suffix}_{current_time}.png")

#### For Visualizing Loss

In [ ]:
def VisualizeLoss(Loss_Arr, folder, epoch, generator = True, SAVE = True):
    x=(range(0,len(Loss_Arr)))
    plt.figure(figsize = (12,10))
    plt.plot(x,Loss_Arr)
    str = "Discriminator"
    if generator:
        str = "Generator"
    plt.xlabel("Number of Iterations")
    plt.ylabel(str + " Loss")
    if SAVE:
        plt.savefig(folder + f"/{str}_Loss_After_Epoch_{epoch}.png")
    plt.show()

## Step 6. Initializing The Model

### Defining Some Hyperparameters

In [ ]:
LEARNING_RATE = 2e-4
EPOCHS = 650
LAMBDA = 100 #Discriminator L1 Loss Hyperparameter as Defined in the Pix2Pix Paper 
epoch = 1
BETAS = (0.5,0.999) #Optimizer Hyperparameter as Defined in the Pix2Pix Paper
lossOfDiscriminator = []
lossOfGenerator = []
saveImages = False #To Save Images during visualization

### Functions and Logic for Loading and Saving Checkpoints

In [ ]:
inputFolder = "../input/model-params"
outputFolder = "/kaggle/working"
checkpointPathDiscriminator = inputFolder+"/disc.pth.tar"
checkpointPathGenerator = inputFolder+"/gen.pth.tar"
loadModel = True

In [ ]:
def SaveCheckpoint(model, optimizer, epoch, filename):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "epoch":epoch,
        "DISC_LOSS" : lossOfDiscriminator,
        "GEN_LOSS" : lossOfGenerator
    }
    torch.save(checkpoint, filename)

def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=device)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    global epoch
    global lossOfDiscriminator
    global lossOfGenerator
    epoch = checkpoint["epoch"]
    lossOfDiscriminator = checkpoint["DISC_LOSS"].copy()
    lossOfGenerator = checkpoint["GEN_LOSS"].copy()

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

### Initializing Models

In [ ]:
disc = Discriminator(3).to(device)
gen = Generator(1).to(device)
optimizerForDiscriminator = optim.Adam(disc.parameters(),lr=LEARNING_RATE, betas=BETAS)
optimizerForGenerator = optim.Adam(gen.parameters(),lr=LEARNING_RATE, betas=BETAS)
LossFunction = nn.BCEWithLogitsLoss()
L1_Loss = nn.L1Loss()
#Float 16 Training for faster Training
discriminatorScaler = torch.cuda.amp.GradScaler()
generatorScaler = torch.cuda.amp.GradScaler()

### Loading Previously Saved Checkpoint if Applicable

In [ ]:
if loadModel:
    load_checkpoint(checkpointPathGenerator, gen, optimizerForGenerator, LEARNING_RATE)
    load_checkpoint(checkpointPathDiscriminator, disc, optimizerForDiscriminator, LEARNING_RATE)

In [ ]:
SaveModel = True
checkpointPathDiscriminator = outputFolder+"/disc.pth.tar"
checkpointPathGenerator = outputFolder+"/gen.pth.tar"

## Step 7. Training

In [ ]:
def TrainFunction(disc, gen, loader, opt_disc, opt_gen, l1_loss, BCE_Loss, gen_scaler, disc_scaler):
    loop = tqdm(loader, leave=True)
    for idx, (L, ab) in enumerate(loop):
        L = L.to(device)
        ab = ab.to(device)
        
        # Train Discriminator
        with torch.cuda.amp.autocast():
            y_fake = gen(L)
            D_real = disc(torch.concat([L, ab],1))
            D_real_loss = BCE_Loss(D_real, torch.ones_like(D_real))
            D_fake = disc(torch.concat([L, y_fake.detach()],1))
            D_fake_loss = BCE_Loss(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2
            lossOfDiscriminator.append(D_loss.item())
        disc.zero_grad()
        disc_scaler.scale(D_loss).backward()
        disc_scaler.step(opt_disc)
        disc_scaler.update()
        
        # Train generator
        with torch.cuda.amp.autocast():
            D_fake = disc(torch.concat([L, y_fake],1))
            G_fake_loss = BCE_Loss(D_fake, torch.ones_like(D_fake))
            L1 = l1_loss(y_fake, ab) * LAMBDA
            G_loss = G_fake_loss + L1
            lossOfGenerator.append(G_loss.item())

        opt_gen.zero_grad()
        gen_scaler.scale(G_loss).backward()
        gen_scaler.step(opt_gen)
        gen_scaler.update()

In [ ]:
while epoch <= EPOCHS:
    print("\nEpoch",epoch,'\n')
    ShowSamples(gen, val_dl,outputFolder,epoch,saveImages)

    if SaveModel:
        SaveCheckpoint(gen, optimizerForGenerator, epoch, filename=checkpointPathGenerator)
        SaveCheckpoint(disc, optimizerForDiscriminator, epoch, filename=checkpointPathDiscriminator)

    if epoch%2==0:
        print("Generator Loss\n")
        VisualizeLoss(lossOfGenerator,outputFolder,epoch,True,saveImages)
        print("Discriminator Loss\n")
        VisualizeLoss(lossOfDiscriminator,outputFolder,epoch,False,saveImages)

    TrainFunction(disc, gen, train_dl, optimizerForDiscriminator, optimizerForGenerator, L1_Loss, LossFunction, discriminatorScaler, generatorScaler)
    epoch+=1


## Step 8. Visualizing Loss Trajectory

### Generator Loss

In [ ]:
VisualizeLoss(lossOfGenerator,outputFolder,epoch,True,SAVE=False)

### Discriminator Loss

In [ ]:
VisualizeLoss(lossOfDiscriminator,outputFolder,epoch,False,SAVE=False)

## Step 9. Visualizing Predictions

### Predictions on Training Data

In [ ]:
numRuns = 5 #Generate numRuns*5 Samples

In [ ]:
for run in range(numRuns):
    ShowSamples(gen, train_dl,outputFolder,epoch,SAVE=True,suffix="_On_Training_Set")

### Predictions on Validation Data

In [ ]:
for run in range(numRuns):
    ShowSamples(gen, val_dl,outputFolder,epoch,SAVE=False)